# Intelligent Agents: Function Execution and Alerts with LLMs


## Introducing Pushover
Setting up Pushover for notifications is quick and hassle-free! Just follow these simple steps:

### Create a Free Account
Head over to https://pushover.net and sign up.

### Generate Your Tokens
You’ll need two tokens to enable notifications:

🔑 User Token – Found on your Pushover dashboard.
🛠️ Application Token – Create one by visiting https://pushover.net/apps/build and registering a new app.
(This allows you to organize notifications into separate apps later.)

Add to Your .env File
Store your credentials securely by adding the following to your .env:

PUSHOVER_USER=your_user_token_here

PUSHOVER_TOKEN=your_app_token_here

### Install the Mobile App
Download the Pushover app on your phone (iOS/Android) to start receiving real-time push notifications.


In [1]:
from dotenv import load_dotenv
from pypdf import PdfReader
import requests
import json
from IPython.display import Markdown, display
import gradio as gr
from pathlib import Path
import os

#Settings
openrouter_base_url="https://openrouter.ai/api/v1"
model="z-ai/glm-4.5-air"

c:\Users\marvi\workspace\agentic_ai\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Send the Mobile Notifications
PUSH_NOTIFICATION_URI="https://api.pushover.net/1/messages.json"
load_dotenv(override=True)

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")

def push_notification(title,message):
    data={
        "token": pushover_token,
        "user": pushover_user,
        "message": message,
        "title": title,
    }

    response = requests.post(PUSH_NOTIFICATION_URI, data)
    if response.status_code == 200:
        return "Notification sent!"
    else:
        return f"Failed to send: {response.text}"

push_notification("Test Mobile Notification", "Testing via API calls")


'Notification sent!'

In [3]:
# Define the tool functions


def record_user_details(email,name="Name not provided",notes="notes not provided"):
    push_notification("user record",f"[User Interest] {name} {email} | Notes: {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push_notification("unknown question",f"[Unknown Question] {question}")
    return {"recorded": "ok"}

# ---- Tool Schemas ----
record_user_details_json = {
    "name": "record_user_details",
    "description": "Record a user's interest using their email and optional details.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {"type": "string", "description": "User's email address"},
            "name": {"type": "string", "description": "User's name"},
            "notes": {"type": "string", "description": "Additional context or comments"},
        },
        "required": ["email"],
        "additionalProperties": False,
    },
}

record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Log a question that the assistant couldn't answer.",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {"type": "string", "description": "The unanswerable question"},
        },
        "required": ["question"],
        "additionalProperties": False,
    },
}

## Tool Dispatcher
Receives tool calls from the LLM (usually via OpenAI’s function-calling feature)

Executes the correct Python function

Returns the result in a format the model expect

You can think of it like a function router for the model's requests:

“The model says: ‘Call record_user_details with this info.’

Your code says: ‘Got it! Let me run that Python function and return the result.’”

In [4]:
# ---- Tool Dispatcher ----

TOOL_FUNCTIONS = {
    "record_user_details": record_user_details,
    "record_unknown_question": record_unknown_question
}

def dispatch_tool_calls(tool_calls):
    """Execute tool calls and return their results."""
    tool_messages = []

    for call in tool_calls:
        # Handle both dict and OpenAI-style object
        if isinstance(call, dict):
            name = call.get("tool")
            args = call.get("arguments", {})
            call_id = call.get("id", "manual-call")
        else:
            name = call.function.name
            args = json.loads(call.function.arguments)
            call_id = getattr(call, "id", "manual-call")

        func = TOOL_FUNCTIONS.get(name)

        if func:
            try:
                tool_result = func(**args)
            except Exception as e:
                tool_result = {"error": str(e)}
        else:
            tool_result = {"error": f"unknown tool {name}"}

        tool_messages.append({
            "role": "tool",
            "content": json.dumps(tool_result),
            "tool_call_id": call_id
        })

    return tool_messages

In [5]:
# Test dispatch_tool_calls function
from types import SimpleNamespace

# --- Mock Tool Call Class (similar to OpenAI's tool_call format) ---
def make_tool_call(name, arguments, call_id="test-id"):
    return SimpleNamespace(
        function=SimpleNamespace(name=name, arguments=json.dumps(arguments)),
        id=call_id
    )

# --- Positive Test Case ---
tool_calls_positive =[
    make_tool_call("record_user_details", {"email": "anshulc55@icloud.com", "name": "Anshul Chauhan"}),
    make_tool_call("record_unknown_question", {"question": "What is the meaning of life?"}),
]

# --- Negative Test Case ---
tool_calls_negative = [
    make_tool_call("non_existent_tool", {"Dummy": "Test Dummy Value"}),
    make_tool_call("record_user_details", {})  # missing required "email"
]

# --- Run Tests ---
print("\n--- Positive Test Case ---")
print(dispatch_tool_calls(tool_calls_positive))

print("\n--- Negative Test Case ---")
print(dispatch_tool_calls(tool_calls_negative))


--- Positive Test Case ---
[{'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'test-id'}, {'role': 'tool', 'content': '{"recorded": "ok"}', 'tool_call_id': 'test-id'}]

--- Negative Test Case ---
[{'role': 'tool', 'content': '{"error": "unknown tool non_existent_tool"}', 'tool_call_id': 'test-id'}, {'role': 'tool', 'content': '{"error": "record_user_details() missing 1 required positional argument: \'email\'"}', 'tool_call_id': 'test-id'}]


In [6]:
# Read the Profile PDF
pdfReader = PdfReader("..\\resources\\Profile.pdf")
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text

        import os

# Get the directory of the current script
script_dir = Path.cwd().parent

# Build the relative path from the script's directory
summ_filePath = os.path.join(script_dir, "resources", "Summary.txt")
with open(summ_filePath, "r", encoding="utf-8") as f:
    summary = f.read()

print(f"File path used: {summ_filePath}")  # Very helpful for debugging!

File path used: c:\Users\marvi\workspace\agentic_ai\resources\Summary.txt


In [7]:
# System Prompt
name = "Sora"
system_prompt = (
    f"You are acting as {name}, representing {name} on their website. "
    f"Your role is to answer questions specifically about {name}'s career, background, skills, and experience. "
    f"You must faithfully and accurately portray {name} in all interactions. "
    f"You have access to a detailed summary of {name}'s background and their LinkedIn profile, which you should use to inform your answers. "
    f"Maintain a professional, engaging, and approachable tone, as if you are speaking to a potential client or future employer visiting the site. "
    f"Always record any unanswered questions using the record_unknown_question tool — even if the question seems minor or unrelated."
    f"If the user shows interest or continues chatting, encourage them to share their email address. Then, use the record_user_details tool to capture their email, name (if provided), and any context worth preserving."

    f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{prof_summary}\n\n"
    f"Using this context, please converse naturally and consistently, always staying in character as {name}."
)

In [56]:
import ollama

def chat_with_tools_ollama(message, model, history, tools):
    # Add the new user message
    messages = [{"role": "user", "content": message}]

    # First model call
    response = ollama.chat(
        model=model,
        messages=messages,
        tools=tools
    )

    assistant_message = response["message"]

    # If the model called a tool
    if assistant_message.get("tool_calls"):
        for tool_call in assistant_message["tool_calls"]:
            name = tool_call["function"]["name"]

            # Handle both string and dict arguments
            raw_args = tool_call["function"]["arguments"]
            if isinstance(raw_args, str):
                args = json.loads(raw_args)
            else:
                args = raw_args

            func = TOOL_FUNCTIONS.get(name)
            if func:
                result = func(**args)

                # Append tool result
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.get("id", "manual-call"),
                    "content": json.dumps(result)
                })

                # Follow-up call after tool execution
                followup = ollama.chat(
                    model=model,
                    messages=messages
                )

                # ✅ Return ONLY the follow-up message
                return followup["message"]["content"]

        return "Sorry, I couldn't execute the requested tool."

    # ✅ If no tool was called, return only the assistant's text
    return assistant_message.get("content", "")

In [59]:
import json
import ollama
import gradio as gr

# --- Tool Functions ---
def record_user_details(email, name="Name not provided", notes="notes not provided"):
    push_notification("user record", f"[User Interest] {name} {email} | Notes: {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push_notification("unknown question", f"[Unknown Question] {question}")
    return {"recorded": "ok"}

# --- Tool Registry ---
TOOL_FUNCTIONS = {
    "record_user_details": record_user_details,
    "record_unknown_question": record_unknown_question
}

# --- Tool Schemas ---
record_user_details_tool = {
    "type": "function",
    "function": {
        "name": "record_user_details",
        "description": "Record a user's interest using their email and optional details.",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {"type": "string", "description": "User's email address"},
                "name": {"type": "string", "description": "User's name"},
                "notes": {"type": "string", "description": "Additional context or comments"},
            },
            "required": ["email"],
            "additionalProperties": False,
        },
    },
}

record_unknown_question_tool = {
    "type": "function",
    "function": {
        "name": "record_unknown_question",
        "description": "Log a question that the assistant couldn't answer.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {"type": "string", "description": "The unanswerable question"},
            },
            "required": ["question"],
            "additionalProperties": False,
        },
    },
}

tools = [record_user_details_tool, record_unknown_question_tool]

def chat_with_tools_ollama(message, model, history, tools):
    # Build complete message history with system prompt
    messages = [{"role": "system", "content": system_prompt}]
    
    # Add conversation history
    messages.extend(history)
    
    # Add the new user message
    messages.append({"role": "user", "content": message})

    # First model call
    response = ollama.chat(
        model=model,
        messages=messages,
        tools=tools
    )

    assistant_message = response["message"]
    
    # Add the assistant's message to the conversation
    messages.append(assistant_message)

    # If the model called a tool
    if assistant_message.get("tool_calls"):
        for tool_call in assistant_message["tool_calls"]:
            name = tool_call["function"]["name"]

            # Handle both string and dict arguments
            raw_args = tool_call["function"]["arguments"]
            if isinstance(raw_args, str):
                args = json.loads(raw_args)
            else:
                args = raw_args

            func = TOOL_FUNCTIONS.get(name)
            if func:
                try:
                    result = func(**args)
                except Exception as e:
                    result = {"error": str(e)}

                # Append tool result to messages
                messages.append({
                    "role": "tool",
                    "content": json.dumps(result)
                })

        # Follow-up call after tool execution with complete context
        followup = ollama.chat(
            model=model,
            messages=messages
        )

        # Return ONLY the follow-up message content
        return followup["message"]["content"]

    # If no tool was called, return only the assistant's text
    return assistant_message.get("content", "")

# --- Gradio Wrapper ---
def gradio_chat(message, history):
    reply = chat_with_tools_ollama(
        message=message,
        model="qwen3:0.6b",  # Use a model that supports tool calling
        history=history,
        tools=tools
    )
    return reply

# Launch the interface
gr.ChatInterface(fn=gradio_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.
